In [2]:
import time
import numpy as np
import pandas as pd
import keras
from keras.engine.topology import Layer

Using TensorFlow backend.
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py

In [3]:
def mape_loss_func(preds, labels):
    mask = labels > 5
    return np.mean(np.fabs(labels[mask]-preds[mask])/labels[mask])

def smape_loss_func(preds, labels):
    mask= labels > 5
    return np.mean(2*np.fabs(labels[mask]-preds[mask])/(np.fabs(labels[mask])+np.fabs(preds[mask])))

def mae_loss_func(preds, labels):
    mask= labels > 5
    return np.fabs((labels[mask]-preds[mask])).mean()

def eliminate_nan(b):
    a = np.array(b)
    c = a[~np.isnan(a)]
    return c

In [4]:
randseed = 25
res = 11

v = pd.read_csv('../data/v_20_aggragated.csv')
v = v.rename(columns={'Unnamed: 0': 'id'})
det_with_class = pd.read_csv('../res/%i_res%i_id_402_withclass.csv'%(randseed, res), index_col=0)

v['class_i'] = ''
for i in range(len(v)):
    v.loc[i, 'class_i'] = det_with_class[det_with_class['id']==v.loc[i, 'id']].iloc[0, 5]  # 5 stands for 'class_i'

num_class = det_with_class['class_i'].drop_duplicates().size

v_class = []
for i in range(num_class):
    v_class.append(v[v['class_i']==i])

print('There are %i class(es)'%num_class)

dist_mat = pd.read_csv('../data/dist_mat.csv', index_col=0)
id_info = pd.read_csv('../data/id2000.csv', index_col=0)
dist_mat.index = id_info['id2']
dist_mat.columns = id_info['id2']
for i in range(len(dist_mat)):
    for j in range(len(dist_mat)):
        if i==j:
            dist_mat.iloc[i, j] = 0

near_id = pd.DataFrame(np.argsort(np.array(dist_mat)), index = id_info['id2'], columns = id_info['id2'])

def get_node(det, seg):
    # det is one single detector id
    # node is one single node id
    
    # seg = pd.read_csv('./data/segement.csv', header=None)
    try:
        node_info = seg[seg[6]==det]
        node = node_info.iloc[0, 0]
    except:
        node_info = seg[seg[7]==det]
        node = node_info.iloc[0, 0]
        
    return node

def get_class_with_node(seg, v_class):
    det_list_class = np.array([])
    try:
        v_class.insert(1, 'id2', '')  # id2 mean node id
    except:
        v_class['id2'] = ''
        
    for i in range(len(v_class)):
        det_list_class = np.append(det_list_class, v_class.iloc[i, 0])
        v_class.iloc[i, 1] = get_node(v_class.iloc[i, 0], seg)
    
    return det_list_class, v_class

def rds_mat(old_dist_mat, det_ids):
    # get a matrix that contains n raods that have specified node id s
    node_ids = np.array([])
    for i in det_ids:
        node_ids = np.append(node_ids, get_node(i, seg))
        
    new_dist_mat = old_dist_mat.loc[node_ids, node_ids]
    old_dist_mat = np.array(old_dist_mat)
    new_near_id_mat = np.argsort(new_dist_mat)
    return new_near_id_mat

seg = pd.read_csv('../data/segement.csv', header=None)
num_dets = 25

det_list_class = []
for i in range(num_class):
    det_list_class_temp, v_class_temp = get_class_with_node(seg, v_class[i])
    det_list_class.append(det_list_class_temp)
    v_class_temp = v_class_temp[v_class_temp['id'].isin(det_list_class_temp[:num_dets])]
    v_class[i] = v_class_temp
    
near_road_set = []
for i in range(num_class):
    near_road_set.append(rds_mat(dist_mat, det_list_class[i][:num_dets]))

There are 5 class(es)


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [5]:
class_i = 3
near_road = np.array(near_road_set[class_i])
flow = np.array(v_class[class_i].iloc[:, 2:-1])

In [6]:
flow.shape

(25, 4464)

In [5]:
# 利用滑动窗口的方式，重构数据为(n，最近路段数，输入时间窗，总路段数)的形式

k = 5 # 参数k为需考虑的最近路段数
t_p = 24 # 参数t_p为总时间序列长度（天）
t_input = 12 #参数t_input为输入时间窗(10min颗粒度)
t_pre = 3 #参数t_pre为预测时间窗(10min颗粒度)
num_links = 25 #参数num_links为总路段数


image = []
for i in range(np.shape(near_road)[0]):
    road_id = []
    for j in range(k):
        road_id.append(near_road[i][j])
    image.append(flow[road_id, :])
image1 = np.reshape(image, [-1, k, len(flow[0,:])])
image2 = np.transpose(image1,(1,2,0))
image3 = []
label = []
day = []

for i in range(1,t_p):
    for j in range(180-t_input-t_pre):
        image3.append(image2[:, i*180+j:i*180+j+t_input, :][:])
        label.append(flow[:, i*180+j+t_input:i*180+j+t_input+t_pre][:])
        day.append(flow[:, (i-1)*180+j+t_input:(i-1)*180+j+t_input+t_pre][:])

image3 = np.asarray(image3)
label = np.asarray(label)
day =  np.asarray(day)

print(np.shape(image3))
print(np.shape(label))
print(np.shape(day))

#划分前80%数据为训练集，最后20%数据为测试集
image_train = image3[:np.shape(image3)[0]*4//5]
image_test = image3[np.shape(image3)[0]*4//5:]
label_train = label[:np.shape(label)[0]*4//5]
label_test = label[np.shape(label)[0]*4//5:]

day_train = day[:np.shape(day)[0]*4//5]
day_test = day[np.shape(day)[0]*4//5:]


(3795, 5, 12, 25)
(3795, 25, 3)
(3795, 25, 3)


In [6]:
# define merge layer
class Merge_Layer(Layer):
    def __init__(self, **kwargs):
        super(Merge_Layer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.para1 = self.add_weight(shape=(input_shape[0][1], input_shape[0][2]),
                                     initializer='uniform', trainable=True,
                                     name='para1')
        self.para2 = self.add_weight(shape=(input_shape[1][1], input_shape[1][2]),
                                     initializer='uniform', trainable=True,
                                     name='para2')
        super(Merge_Layer, self).build(input_shape)

    def call(self, inputs):
        mat1 = inputs[0]
        mat2 = inputs[1]
        output = mat1 * self.para1 + mat2 * self.para2
        # output = mat1 * 0.1 + mat2 * 0.9
        return output

    def compute_output_shape(self, input_shape):
        return input_shape[0]

In [7]:
input_data = keras.Input(shape=(k,t_input,num_links), name='input_data')
input_HA = keras.Input(shape=(num_links, t_pre), name='input_HA')

x = keras.layers.BatchNormalization(input_shape =(k,t_input,num_links))(input_data)

x = keras.layers.Conv2D(
                           filters = num_links,
                           kernel_size = 3,
                           strides = 1,
                           padding="SAME",
                           activation='relu')(x)

x = keras.layers.AveragePooling2D(pool_size = (2,2),
                                strides = 1,
                                padding = "SAME",
                                )(x)

x = keras.layers.BatchNormalization()(x)

x = keras.layers.Conv2D(
                       filters = num_links,
                       kernel_size = 3,
                       strides = 1,
                       padding="SAME",
                       activation='relu')(x)

x = keras.layers.AveragePooling2D(pool_size = (2,2),
                                strides = 1,
                                padding = "SAME",
                                )(x)
x = keras.layers.Flatten()(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dropout(0.5)(x)

x = keras.layers.Dense(num_links*2*t_pre, activation='relu', name='dense_1')(x)
x = keras.layers.Dense(num_links*t_pre, activation='relu', name='dense_2')(x)

output = keras.layers.Reshape((num_links,t_pre))(x)

output_final = Merge_Layer()([output, input_HA])

# construct model
finish_model = keras.models.Model([input_data,input_HA], [output_final])

finish_model.summary()








Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_data (InputLayer)         (None, 5, 12, 25)    0                                            
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 5, 12, 25)    100         input_data[0][0]                 
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 5, 12, 25)    5650        batch_normalization_1[0][0]      
__________________________________________________________________________________________________
average_pooling2d_1 (AveragePoo (None, 5, 12, 25)    0           conv2d_1[0][0]       

In [8]:
finish_model.compile(optimizer='adam', loss='mean_squared_error')

In [9]:
X_train = image_train
X_HA_train = day_train
label_train = label_train

In [10]:
#模型拟合与评估
finish_model.fit([X_train,X_HA_train], label_train, epochs=100, batch_size=128,
validation_data=([image_test,day_test], label_test))
# finish_model.evaluate(image_test, label_test)

Train on 3036 samples, validate on 759 samples
Epoch 1/100
3036/3036 [==============================] - 1s 472us/step - loss: 3113.6255 - val_loss: 2879.2247
Epoch 2/100
3036/3036 [==============================] - 0s 70us/step - loss: 2626.9172 - val_loss: 2035.6383
Epoch 3/100
3036/3036 [==============================] - 0s 68us/step - loss: 1881.8811 - val_loss: 1637.0610
Epoch 4/100
3036/3036 [==============================] - 0s 66us/step - loss: 1571.0849 - val_loss: 1460.4864
Epoch 5/100
3036/3036 [==============================] - 0s 67us/step - loss: 1402.6526 - val_loss: 1351.1354
Epoch 6/100
3036/3036 [==============================] - 0s 67us/step - loss: 1270.3794 - val_loss: 1218.9148
Epoch 7/100
3036/3036 [==============================] - 0s 66us/step - loss: 1179.6136 - val_loss: 1140.0141
Epoch 8/100
3036/3036 [==============================] - 0s 68us/step - loss: 1115.0033 - val_loss: 1078.8210
Epoch 9/100
3036/3036 [==============================] - 0s 66us/step - 

3036/3036 [==============================] - 0s 71us/step - loss: 209.8597 - val_loss: 217.2645
Epoch 76/100
3036/3036 [==============================] - 0s 70us/step - loss: 210.2788 - val_loss: 217.2577
Epoch 77/100
3036/3036 [==============================] - 0s 72us/step - loss: 209.5327 - val_loss: 216.2211
Epoch 78/100
3036/3036 [==============================] - 0s 72us/step - loss: 208.5433 - val_loss: 215.7607
Epoch 79/100
3036/3036 [==============================] - 0s 70us/step - loss: 208.5522 - val_loss: 215.5557
Epoch 80/100
3036/3036 [==============================] - 0s 70us/step - loss: 207.1860 - val_loss: 215.0101
Epoch 81/100
3036/3036 [==============================] - 0s 69us/step - loss: 207.0284 - val_loss: 214.9917
Epoch 82/100
3036/3036 [==============================] - 0s 71us/step - loss: 206.8377 - val_loss: 215.4845
Epoch 83/100
3036/3036 [==============================] - 0s 73us/step - loss: 208.0434 - val_loss: 214.6611
Epoch 84/100
3036/3036 [========

In [11]:
#模型预测
model_pre = finish_model.predict([image_test,day_test])

In [12]:
#计算各项误差指标

mape_mean = mape_loss_func(model_pre, label_test)
smape_mean = smape_loss_func(model_pre, label_test)
mae_mean = mae_loss_func(model_pre, label_test)

print('mape = ' + str(mape_mean) + '\n' + 'smape = ' + str(smape_mean) + '\n' + 'mae = ' + str(mae_mean))

mape = 0.23380487718881862
smape = 0.22049753617084464
mae = 9.6413509935195


In [13]:
#模型保存
finish_model.save_weights('../model/source_fulldata.h5')


In [14]:
#计算每条路段的误差
mape_list = []
for i in range(num_links):
    a1 = mape_loss_func(model_pre[:,i,:], label_test[:,i,:])
    mape_list.append(a1)
    print(str(i+1)+'th link')
    print(a1)

1th link
0.3849126232443209
2th link
0.09890990970882008
3th link
0.3779040752870145
4th link
0.24624090079132263
5th link
0.16636187383961043
6th link
0.11274875592974792
7th link
0.13098273509911168
8th link
0.0404700803400431
9th link
0.10018476130168177
10th link
0.17079389852769958
11th link
0.10038331805295737
12th link
0.05503035144440833
13th link
0.217240494239185
14th link
0.39741582274293175
15th link
0.27841386279939384
16th link
0.2190412534007548
17th link
0.3390328380219796
18th link
0.11354546593447948
19th link
0.15398636594382736
20th link
0.15089653580255932
21th link
0.16309076102579817
22th link
0.23956478431472472
23th link
0.5401646229274
24th link
0.6897578533766308
25th link
0.36018318396677546


In [15]:
mape_pd = pd.Series(mape_list)

In [16]:
mape_pd.sort_values()

7     0.040470
11    0.055030
1     0.098910
8     0.100185
10    0.100383
5     0.112749
17    0.113545
6     0.130983
19    0.150897
18    0.153986
20    0.163091
4     0.166362
9     0.170794
12    0.217240
15    0.219041
21    0.239565
3     0.246241
14    0.278414
16    0.339033
24    0.360183
2     0.377904
0     0.384913
13    0.397416
22    0.540165
23    0.689758
dtype: float64